In [12]:
from ortools.linear_solver import pywraplp
solver=pywraplp.Solver.CreateSolver('SCIP')
def create_model():
    data={}
    data['distance']=[[0,125,225,155,215],
                      [125,0,85,115,135],
                      [225,85,0,165,190],
                      [155,112,165,0,195],
                      [215,135,190,195,0]]
    data['city']=['Basin','Wald','Bon','Mena','Kiln']
    data['vehicle']=1
    data['depot']=0
    return data
    
def print_solution(data,Data,x,status):
    if status==pywraplp.Solver.OPTIMAL:
        print(f'Total distance = {solver.Objective().Value()}')
        for i in range(Data):
            for j in range(Data):
                if i!=j and x[i,j].solution_value() !=0:
                    print(data['city'][i],'->',data['city'][j])

def main():
    data=create_model()
    x={}
    u={}
    Data=len(data['distance'])
    for i in range(Data):
        u[i]=solver.IntVar(0,solver.infinity(),'u[%i]'%i)
        for j in range(Data):
            if i!=j:
                x[i,j]= solver.BoolVar(f'x[{i},{j}]')
    
    for i in range(Data):
        solver.Add(sum(x[i,j] for j in range(Data) if i!=j)==1)
    for j in range(Data):
        solver.Add(sum(x[i,j] for i in range(Data) if i!=j)==1)
    for i in range(1,Data):
        for j in range(1,Data):
            if i!=j:
                solver.Add(u[i]-u[j]+Data*x[i,j]<=(Data-1))
    solver.Add(u[0]==0)
    obj=[]
    for i in range(Data):
        for j in range(Data):
            if i!=j:
                obj.append(data['distance'][i][j]*x[i,j])
    solver.Minimize(sum(obj))
    status=solver.Solve()
    return print_solution(data,Data,x,status)

if __name__=='__main__':
    main()

Total distance = 750.0
Basin -> Mena
Wald -> Basin
Bon -> Wald
Mena -> Kiln
Kiln -> Bon


In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
def create_model():
    data={}
    M=1000000
    data['distance']=[[0,125,225,155,215],
                      [123,0,85,115,135],
                      [225,85,0,165,190],
                      [155,112,165,0,195],
                      [215,135,190,195,0]]
    data['vehicle']=1
    data['depot']=0
    return data

def print_solution(manager,routing,solution):
    print(f'Total distance : {solution.ObjectiveValue()}')
    index=routing.Start(0)
    plan_output =f'Routing of Vehicle 0 :\n'
    route_distance=0
    while not routing.IsEnd(index):
        plan_output+=f'{manager.IndexToNode(index)}->'
        previous_index=index
        index=solution.Value(routing.NextVar(index))
        route_distance+=routing.GetArcCostForVehicle(
                previous_index,index,0
            )
    plan_output+=f'{manager.IndexToNode(index)}\n'
    plan_output+=f'Distance of Route: {route_distance}'
    print(plan_output)

def main():
    data=create_model()
    manager=pywrapcp.RoutingIndexManager(
        len(data['distance']),data['vehicle'],data['depot']
    )
    routing=pywrapcp.RoutingModel(manager)

    def distance_callback(from_index,to_index):
        from_node=manager.IndexToNode(from_index)
        to_node=manager.IndexToNode(to_index)
        return data['distance'][from_node][to_node]
    transit_callback_index=routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameter=pywrapcp.DefaultRoutingSearchParameters()
    search_parameter.first_solution_strategy= routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC


    solution=routing.SolveWithParameters(search_parameter)
    if solution:
        print_solution(manager,routing,solution)


if __name__=='__main__':
    main()

Total distance : 748
Routing of Vehicle 0 :
0->3->4->2->1->0
Distance of Route: 748
